In [ ]:
from matplotlib import pyplot as plt
from matplotlib.patches import Patch
from matplotlib.colors import to_rgb
from matplotlib.ticker import PercentFormatter, FuncFormatter
from datetime import timedelta
import json

def to_time(t, _):
    m = int(t / 60)
    s = int(t) % 60
    return f"{m:02d}:{s:02d}"

with open("wtf.json") as file:
    data = json.load(file)

In [ ]:
sorted_trivias = [t for t, v in sorted(data["last_values"].items(), key=lambda o: -o[1])]
highscore = sorted_trivias[:8]

for index, trivia in enumerate(highscore, start=1):
    plot = data["plot"][trivia]
    label = f"{index:}. {trivia}"
    x = [p[0] for p in plot]
    y = [p[1] for p in plot]
    plt.plot(x, y, label=label)

ax = plt.gca()
fig = plt.gcf()
fig.set_size_inches(15, 5)
ax.legend()
ax.grid(alpha=0.50, which="major")
ax.grid(alpha=0.25, which="minor")
ax.yaxis.set_major_formatter(PercentFormatter(xmax=1.0))
ax.xaxis.set_major_formatter(FuncFormatter(to_time))

In [ ]:
import numpy as np
import matplotlib.colors as mcolors
from cycler import cycler

cycle = cycler(color=['tab:blue', 'tab:red', 'tab:green', 'yellow', 'cyan', 'fuchsia'])
legend = []
max_v = 1
TOP = 6

for trivia in highscore[:TOP]:
    heatmap = data["heatmap"][trivia]
    max_v = max(max_v, max(max(x for x in y) for y in heatmap))
    
for params, (index, trivia) in zip(cycle, enumerate(highscore[:TOP], start=1)):
    color = to_rgb(params["color"]) + (1.0,)
    heatmap = np.array(data["heatmap"][trivia])
    img = color * (heatmap / max_v).reshape(*heatmap.shape, 1)
    plt.imshow(img)
    legend.append(Patch(facecolor=color, label=f"{index}. {trivia}"))

ax = plt.gca()
ax.set_xticks([])
ax.set_yticks([])
ax.legend(
    handles=legend,
    bbox_to_anchor=(0.5, 0.0),
    loc='upper center',
    ncol=3,
)
fig = plt.gcf()
fig.set_size_inches(img.shape[1] / 3.0, img.shape[0] / 3.0)